In [1]:
import numpy as np
import pandas as pd
!pip install ortools;
import ortools
from ortools.linear_solver import pywraplp
import pickle 

solver = pywraplp.Solver('LinearProgrammingExample',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)


df = pd.read_pickle("dummy.pkl")

In [2]:
vectors = df["Vector"]
distances = np.zeros((2500,2500))
from scipy.spatial import distance

for i in range(len(vectors)):
    a = vectors[i]
    for j in range(len(vectors)):
        b = vectors[j]
        dst = distance.cityblock(a, b)
        distances[i,j] = dst

In [3]:
C = []
authors = df["Author"].unique()

for i in authors:
  df_sub = df[df["Author"] == i]
  C.append(df_sub.index.values)

In [4]:
a = 0.5
a_1 = np.ceil(1/a)
colors = 50
k = 25
lam = 15
n = 2500

In [1]:
x = []
y = []
a = 0.5
a_1 = np.ceil(1/a)
colors = 50
k = 25
#lam = 15
n = 2500
F_small = []

solver = pywraplp.Solver('LP',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

objective = solver.Objective()
for i in range(n):
    x.append([])
    y.append(solver.NumVar(0.0, 1.0,'y'))
    for j in range(n):
        x[i].append(solver.NumVar(0.0, 1.0,'x'))
        objective.SetCoefficient(x[i][j], distances[i][j])
objective.SetMinimization()

constraint = []
for j in range(n):
    constraint.append(solver.Constraint(1, 1))
    for i in range(n):
        constraint[j].SetCoefficient(x[i][j],1)

for i in range(n):
    for j in range(n):
        constraint.append(solver.Constraint(-solver.infinity(), 0))
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-1)

for i in range(colors):
    constraint.append(solver.Constraint(-solver.infinity(), 0))
    for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],-a)
    if j in C[i]:
        constraint[-1].SetCoefficient(x[i][j],1-a)

for i in range(n):
    constraint.append(solver.Constraint(0, solver.infinity()))
    for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-a_1)

constraint.append(solver.Constraint(0, k))
for i in range(n):
    constraint[-1].SetCoefficient(y[i],1)

#for i in range(n):
#    for j in range(n):
#        if distances[i,j] > lam:
#            constraint.append(solver.Constraint(0, 0))
#            constraint[-1].SetCoefficient(x[i][j],1)

status = solver.Solve()
status == solver.OPTIMAL

NameError: name 'np' is not defined